# Introduction
This tutorial will introduce you to a powerful Python interactive data visualization library, Bokeh. 

Main characteristics of Bokeh:
- interactive
- high-performance on a big data set
- easy to use
- fancy data visualization for presentation

First sight of Bokeh, TED speech [A REPRODUCTION OF GAPMINDER](https://demo.bokehplots.com/apps/gapminder)

# Tutorial content
This tutorial is aimed to provide you a quick start of Bokeh library. 

For easy to use, Bokeh classifies their interfaces into three levels. In this tutorial, I would like to introduce them in the following order which I believe is matched to its hardness, from easy to difficult.
- bokeh.plotting: an intermediate-level interface to draw plots like line, circle
- bokeh.charts: a high-level interface to build statistical charts such as scatter, histogram, and whisker
- bokeh.models: a low-level interface for application developers to create their own Bokeh object and server


We will cover the following topics in this tutorial:
1. Installing the libraries
2. Authentication
3. Loading data and plotting (bokeh.plotting: Line)
4. Data processing
5. Making high-level charts (bokeh.charts: Scatter)
6. Working on low-level interface (bokeh.models: Map)
7. Running a Bokeh Server

# Installing the libraries
Before starting, you need to install some libraries that we will use. Apparently, we need to install bokeh. Besides, we will use tweepy to interact with Twitter API. It should be easy for students in 15388/15688 to install these libraries via pip command.

    $pip install bokeh
$pip install tweepy

For students out of the class, you may lack of some required dependencies. The following command would help you install all of them.

    $conda install bokeh

Now, you can play with the libraries. Enjoy it!

In [5]:
import io, time, json 
import pandas as pd
import tweepy #import twitter python api library

# Authentication

Do you remember OAuth we learned in homework 1? We generate a token on Yelp API Website then create an authenticated yelp-python client. Twitter also uses this technology to control access to their API. Actually, this protocol was raised by a former developer of Twitter, Blaine Cook, when he interacted with Twitter API. Now, let us recall the accessing process. First, you need a valid token. Then, call OAuth interfaces to get an authenticated api instance.

## Twitter API Access Steps:
1. Create a Twitter account if you don't have
2. Generate your own token [here](https://apps.twitter.com)
3. Save your consumer key and access token in a local file in json format
4. Create an authenticated twitter-python api

Json Template:
{
    "consumer_key": "XXX",
    "consumer_secret": "XXX",
    "access_token": "XXX",
    "access_token_secret": "XXX"
}


In [6]:
def authenticate(config_filepath):
    """
    Create an authenticated twitter-python api.

    Args:
        config_filepath (string): relative path (from this file) to a file with your twitter credentials

    Returns:
        api (tweepy.api.API): authenticated instance of a tweepy api
    """
    with io.open(config_filepath, 'r') as config_secret:
        creds = json.load(config_secret)
        
        auth = tweepy.OAuthHandler(creds['consumer_key'], creds['consumer_secret'])
        auth.set_access_token(creds['access_token'], creds['access_token_secret'])
        return tweepy.API(auth)
    pass

In [7]:
api = authenticate('config_secret.json')

#Now you could do some simple tests to check if the tweepy api works
print "user_name:", api.me().name
print "followers_count:", api.me().followers_count

user_name: J
followers_count: 20


# Loading data and plotting
The Battery of Election 2016 is more and more intensive and interesting. I am sure you want to know who is more popular on Twitter, Trump or Hillary? Let's load the latest 20 tweets of Trump and Hillary for our first Bokeh plot. 

First of all, you need to gather tweets of a specific user and load into a pandas dataframe. You may think user_timeline is helpful. [tweepy API Reference](http://docs.tweepy.org/en/v3.5.0/api.html)

In [8]:
# load Trump's tweets into pandas dataframe
trump_tweets = []
for trump_tweet in api.user_timeline('@realDonaldTrump'):
    tweet_dict = {'user_id':trump_tweet.user.id,\
                  'user_name':trump_tweet.user.name, \
                  'user_statuses_count':trump_tweet.user.statuses_count,\
                  'user_followers_count':trump_tweet.user.followers_count,\
                  
                  'tweet_created_at':trump_tweet.created_at,\
                  'tweet_text':trump_tweet.text,\
                  'tweet_retweet_count':trump_tweet.retweet_count,\
                  'tweet_favorite_count':trump_tweet.favorite_count}
    trump_tweets.append(tweet_dict)
    
trump_tweets_df = pd.DataFrame(data = trump_tweets)
print trump_tweets_df.head()

     tweet_created_at  tweet_favorite_count  tweet_retweet_count  \
0 2016-11-02 03:43:15                  3025                 1247   
1 2016-11-02 00:32:51                 17489                 6651   
2 2016-11-01 23:07:25                     0                 2134   
3 2016-11-01 21:57:54                 17587                 9729   
4 2016-11-01 21:46:18                 11540                 5713   

                                          tweet_text  user_followers_count  \
0  Join me in Florida tomorrow!\n\nMIAMI•12pm\nht...              12859734   
1  Thank you for your incredible support Wisconsi...              12859734   
2  RT @DanScavino: Join @realDonaldTrump LIVE in ...              12859734   
3  WikiLeaks emails reveal Podesta urging Clinton...              12859734   
4  'Podesta urged Clinton team to hand over email...              12859734   

    user_id        user_name  user_statuses_count  
0  25073877  Donald J. Trump                33888  
1  25073877  Donal

In [9]:
# load Hillary's tweets into pandas dataframe
clinton_tweets = []
for clinton_tweet in api.user_timeline('@HillaryClinton'):
    tweet_dict = {'user_id':clinton_tweet.user.id,\
                  'user_name':clinton_tweet.user.name, \
                  'user_statuses_count':clinton_tweet.user.statuses_count,\
                  'user_followers_count':clinton_tweet.user.followers_count,\
                  
                  'tweet_created_at':clinton_tweet.created_at,\
                  'tweet_text':clinton_tweet.text,\
                  'tweet_retweet_count':clinton_tweet.retweet_count,\
                  'tweet_favorite_count':clinton_tweet.favorite_count}
    clinton_tweets.append(tweet_dict)
    
clinton_tweets_df = pd.DataFrame(data = clinton_tweets)
print clinton_tweets_df.head()

     tweet_created_at  tweet_favorite_count  tweet_retweet_count  \
0 2016-11-02 02:11:23                  4063                 1563   
1 2016-11-02 01:29:47                  3995                 2891   
2 2016-11-02 01:06:52                  2810                 1065   
3 2016-11-02 00:00:27                  5859                 2763   
4 2016-11-01 23:10:38                     0                  886   

                                          tweet_text  user_followers_count  \
0  Fighting for kids in foster care has been one ...              10101939   
1  Election Day is in one week—and once it's over...              10101939   
2  More people will early vote in 2016 than ever ...              10101939   
3  Be like @JoeBiden:\n\nAlways make a plan. http...              10101939   
4  RT @RobbyMook: Some stats from the field: door...              10101939   

      user_id        user_name  user_statuses_count  
0  1339835893  Hillary Clinton                 9521  
1  1339835893 

Now, it's time to show your data using bokeh.plotting. Basically, there are two steps. First, configure the attributes of the figure, like plot size, plot title, and axis label. Second, draw the line. As matplotlib, you can draw multiple lines to show on the same plot.

In general, Bokeh would generate a html page for displaying the output. e.g. `output_file("xxx.html")` But, Bokeh supports jupyter officially, so there is another useful approach named `output_notebook()` could generate plots directly on jupyter notebook for presentation.


In [13]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import HoverTool

output_notebook() # display the output in jupyter

Loading BokehJS ...

In [12]:
# plot configuration
TOOLS = "pan,box_zoom,wheel_zoom,reset,save" # configure the tool bar
lines_p = figure(tools=TOOLS, \
           plot_width=400, plot_height=400, \
           title="Trump vs Clinton", \
           y_axis_label='Retweet Count')

# add lines. Yes, Bokeh support pandas dataframe
lines_p.line(trump_tweets_df.index, trump_tweets_df.loc[:,'tweet_retweet_count'], \
       legend='Trump', line_width=2, line_color='navy')
lines_p.line(clinton_tweets_df.index, clinton_tweets_df.loc[:,'tweet_retweet_count'], \
       legend='Clinton', line_width=2, line_color='orange')

show(lines_p)

As most the students in 15388/15688 class have experience on matplot, you may ask what's the difference between matplot and Bokeh? The interface and the output plot looks similar. 

Now, please try the functions, like wheel zoom, on the tool bar which is located on the top right corner of the plot. Then you would understand the main feature of Boleh is interactive.

## Styling and embellish
According to above plot, it seems that tweets of Trump are retweeted more frequently than Hillary in recent days. In order to make the plot more clear to the audience, Bokeh provides many ways to highlight the data. For example, you could use shapes like circle, triangle or square in different colors to show the data. Also, you can see concrete data number in hover when you move your mouse on the circle.

More layout information in [Styling Visual Attributes](http://bokeh.pydata.org/en/latest/docs/user_guide/styling.html#using-palettes) could help you create a fancy plot for sharing.

In [14]:
lines_p.add_tools(HoverTool()) # add hover tool into tool bar, do not add it twice!

In [16]:
# highlight your data using circle shape
lines_p.circle(trump_tweets_df.index, trump_tweets_df.loc[:,'tweet_retweet_count'], \
         legend='Trump', fill_color="navy", line_color="navy", size=10)
lines_p.circle(clinton_tweets_df.index, clinton_tweets_df.loc[:,'tweet_retweet_count'], \
         legend='Clinton', fill_color="orange", line_color="orange", size=10)

# configure hover tool
hover = lines_p.select_one(HoverTool)
hover.mode = "mouse" # consider the point position as a point or span or direction
hover.tooltips = [
    ("Retweet Counts", "@y") # hover information: attribute name, data
]

show(lines_p)

It should be more clearly now. I am sure Trump must be a Social queen. Also, it is heard he want to open his own media company. 

# Data processing
Before learning advance Bokeh interfaces, we need to prepare a data set for US 2016 election poll investigation. In the next three examples, we will use the same data set.

Using search api, we can get the most recent 2500 tweets which contain the key words votetrump or votehillary. Then we will group by the data basing on their geo information (USA state) and whom they support (text contains "votetrump" or "votehillary").

In [87]:
tweets = []
# load recent 2500 tweets contains key words: votetrump or votehillary
for tweet in tweepy.Cursor(api.search, q="votetrump%20OR%20votehillary", \
                           show_user = True, result_type = "recent",\
                           since="2016-10-25", until="2016-11-15", lang="en").items(2500):

    tweet_dict = {'user_id':tweet.user.id,\
                  'user_name':tweet.user.name, \
                  'user_statuses_count':tweet.user.statuses_count,\
                  'user_followers_count':tweet.user.followers_count,\
                  
                  'tweet_created_at':tweet.created_at,\
                  'tweet_retweet_count':tweet.retweet_count,\
                  'tweet_favorite_count':tweet.favorite_count,\
                  'tweet_text':tweet.text,\
                  
                  'user_location':tweet.user.location[-2:] # filling location in user profile 
                                                           # may be any word like 'winterfell'
                                                           # only consider correct state code

#                   decide to use user_location since the following attributes are none
                  
                  # tweets sending area information, most values are none
#                   'tweet_country_code':tweet.place.country_code,\
#                   'tweet_place_type':tweet.place.place_type,\
#                   'tweet_state':tweet.place.full_name[-2:],\
                  
                  # tweets sending location, most values are none
#                   'c':tweet.coordinates,\ 
                  }
    
    tweets.append(tweet_dict)

In [88]:
# load into pandas dataframe
vote_tweets_df = pd.DataFrame(data = tweets)
print vote_tweets_df.head()

     tweet_created_at  tweet_favorite_count  tweet_retweet_count  \
0 2016-11-02 19:28:46                     0                    0   
1 2016-11-02 19:28:46                     0                  189   
2 2016-11-02 19:28:44                     0                  208   
3 2016-11-02 19:28:44                     0                    0   
4 2016-11-02 19:28:43                     0                  815   

                                          tweet_text  user_followers_count  \
0  Exactly! Lynch still trying to obstruct justic...                  1818   
1  RT @ofccadjust: Love it 🇺🇸🇺🇸 #MAGA #MakeAm...                 29600   
2  RT @_Makada_: Donald Trump in Miami, FL: "I lo...                   221   
3  How #Clintons destroyed #Haiti #ClintonFoundat...                  2325   
4  RT @Miami4Trump: African American's Are Outrag...                  2929   

              user_id user_location          user_name  user_statuses_count  
0          3389994447            SA     Brandy 4

In [89]:
# clean tweets which mention both Trump and Hillary
vote_clean_df = vote_tweets_df.drop(\
                    vote_tweets_df.loc[:,'tweet_text'].str.contains(\
                    '.*votetrump.*votehillary.*|.*votehillary.*votetrump.*', case=False))

In [90]:
# create an indication for support Trump or Hillary
vote_clean_df['support_trump'] = \
        vote_clean_df.loc[:,'tweet_text'].str.contains('votetrump', case=False)
print vote_clean_df.head()

     tweet_created_at  tweet_favorite_count  tweet_retweet_count  \
1 2016-11-02 19:28:46                     0                  189   
2 2016-11-02 19:28:44                     0                  208   
3 2016-11-02 19:28:44                     0                    0   
4 2016-11-02 19:28:43                     0                  815   
5 2016-11-02 19:28:41                     0                   10   

                                          tweet_text  user_followers_count  \
1  RT @ofccadjust: Love it 🇺🇸🇺🇸 #MAGA #MakeAm...                 29600   
2  RT @_Makada_: Donald Trump in Miami, FL: "I lo...                   221   
3  How #Clintons destroyed #Haiti #ClintonFoundat...                  2325   
4  RT @Miami4Trump: African American's Are Outrag...                  2929   
5  RT @SarahWilsoninCA: #WednesdayWisdom #VoteTru...                  3071   

              user_id user_location          user_name  user_statuses_count  \
1  704034081898668032            SA  Jordan 🐸 M

In [91]:
# in order to relate tweets with USA states
# join user location with state master information
# you can prepare state master info by yourself or download from internet
state_latlon_df = pd.read_csv('state_latlon.csv', na_filter=False)
vote_geo_df = \
    pd.merge(vote_clean_df, state_latlon_df, how='inner', \
             left_on='user_location', right_on='state')
vote_geo_df['tweet_count'] = 1
print vote_geo_df.head()

     tweet_created_at  tweet_favorite_count  tweet_retweet_count  \
0 2016-11-02 19:28:39                     0                    0   
1 2016-11-02 19:28:22                     0                   10   
2 2016-11-02 19:28:17                     0                   30   
3 2016-11-02 19:27:50                     0                    6   
4 2016-11-02 19:27:36                     0                    3   

                                          tweet_text  user_followers_count  \
0  Peter Thiel is dropping truth bomb after truth...                   484   
1  RT @ddindy04: Get to know Huma Abedin\n#podest...                  3579   
2  RT @Stevenwhirsch99: It's extremely disappoint...                    42   
3  RT @LbrtyNow: Hillary hates Trump supporters -...                  3579   
4  RT @McDebida: Due to campaign stress &amp; the...                  3579   

      user_id user_location            user_name  user_statuses_count  \
0    68457801            TX    Lauren Aniess🇺🇸   

In [92]:
# sum tweets, retweets count, favorite count
grouped_vote = \
    vote_geo_df.groupby(['state','support_trump','latitude','longitude'])\
            ['tweet_favorite_count','tweet_retweet_count','tweet_count'].sum()
# reset index
grouped_vote = grouped_vote.reset_index(level=['state','support_trump','latitude','longitude'])

In [93]:
# replace the value in support_trump column, True -> Trump; False -> Hillary
# this info can be used for grouping color on bokeh scatter plot
grouped_vote.loc[:,'support_trump'] = \
        grouped_vote.loc[:,'support_trump'].replace(True, 'Trump')
grouped_vote.loc[:,'support_trump'] = \
        grouped_vote.loc[:,'support_trump'].replace(False, 'Hillary')
print grouped_vote.head()

  state support_trump  latitude  longitude  tweet_favorite_count  \
0    AK       Hillary   61.3850  -152.2683                     0   
1    AK         Trump   61.3850  -152.2683                     0   
2    AL       Hillary   32.7990   -86.8073                     0   
3    AL         Trump   32.7990   -86.8073                    33   
4    AR       Hillary   34.9513   -92.3809                     0   

   tweet_retweet_count  tweet_count  
0                  225            1  
1                  184            2  
2                  225            1  
3                  259            4  
4                  260            2  


## Reflection on data processing
There are three concerns of the data processing.
- accuracy:

It is obviously that 2500 tweets cannot represent 0.3 billion people in the USA. In order to make more accurate analysis, we need a bigger data set. But, twitter has [rate limitation](https://dev.twitter.com/rest/public/rate-limiting) to prevent you consume away their api resources. So, you can add intervals (sleep) between your requests. Besides, twitter api only provides latest tweets within one week. You could turn to third party storage for twitter historical data. 
- language context:

Language is interesting and formidable. Does a tweet which contains "votetrump" mean the sender support trump? What if the tweet is "only jerk would #votetrump"? Thus, advanced text analysis technique could be used in such case to analyze the standpoint of the speaker. This part would not include in this library tutorial.

- data missing:

As the comments on the code, we choose last two alphabets of "user_location" attribute in the user profile as geo-information of the tweet. Actually, user_location is filled by user and it can be any word related or unrelated to geo-location, such as 'Winterfell' (place in a popular TV show) or 'Pittsburgh, PA'. The coordinates (latitude and longitude of the tweet sending position) or place attributes of tweets are more accurate than user_location. But most of these attributes are 'None' due to user privacy protection. So, let's use user_location as an example for this Bokeh library toturial. This tutorial is not focus on data processing and data analytics.


# Making high-level charts
Go back to Bokeh library. Bokeh supports a series of general statistical charts, including Area, Histogram, TimeSeries. Let's take Scatter as an example. Similar to plotting, it is easy to draw a scatter diagram by entering figure information and data information into Scatter interface.

In [94]:
from bokeh.charts import Scatter, output_notebook, show
output_notebook()

Loading BokehJS ...

In [95]:
# scatter can mark data as different colors automatically if you provide color group by column
# in this case, color by column is 'support_trump'
# so, data of trump and hillary has been separated into 2 colors
# color and data value mapping show on the chart
scatter_p = Scatter(grouped_vote, x='tweet_count', y='tweet_retweet_count', \
                    color='support_trump', \
                    plot_width=400, plot_height=400,\
                    title="Trump vs Hillary on Vote Tweets", \
                    xlabel="Tweet Count", ylabel="Retweet Count")
show(scatter_p)

# Working on low-level interface
It seems Trump is more popular in 4 states undoubtedly. What about the other states? The results are unclear from the scatter chart.

So, we can mark who got more retweet count on the USA map with low-level interface.

In [109]:
# make a copy, since grouped_vote is still needed in the last example
grouped_vote_cp = grouped_vote.copy(deep=True)

In [110]:
# group by state. 
# within the same state, using retweet count of trump minus retweet count of hillary.
# if left retweet count is positive => trump state
# if left retweet count is negative => hillary state
grouped_vote_cp.loc[grouped_vote_cp['support_trump'] == 'Hillary', 'tweet_retweet_count'] = \
                grouped_vote_cp.loc[:,'tweet_retweet_count']*-1
state_diff = grouped_vote_cp.groupby(['state'])['tweet_retweet_count'].sum()
state_diff = state_diff.reset_index(level=['state'])
print state_diff.head()

  state  tweet_retweet_count
0    AK                  -41
1    AL                   34
2    AR                  -50
3    AS                 -172
4    AZ                 1880


Since Bokeh color map does not contain states or counties border information, we need to prepare bounding boxes data of each state. 
Instead of downloading or generating bounding boxes online, it is easy to use bounding boxes in Bokeh sample data. Run the following command to get sample data.

    $bokeh sampledata

Or you can try to use GMapPlot (Google Maps Support) after this tutorial.

In [113]:
from bokeh.io import show
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper
)
from bokeh.palettes import RdBu9, RdGy9
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.sampledata import us_states, us_counties, unemployment
output_notebook()

Loading BokehJS ...

In [114]:
us_states = us_states.data.copy()
# do not include Alaska and Hawaii in the map
del us_states["HI"]
del us_states["AK"]

state_xs = [us_states[code]["lons"] for code in us_states] # bounding box
state_ys = [us_states[code]["lats"] for code in us_states] # bounding box

rate_dict = dict(zip(state_diff.loc[:,'state'], state_diff.loc[:,'tweet_retweet_count']))
state_color = []
state_names = []
state_rates = []
for sname in us_states:
    state_names.append(sname)
    try:
        if rate_dict[sname] > 0:
            state_color.append(RdBu9[-2]) # if more retweets of votetrump, mark as red
            state_rates.append(rate_dict[sname])
        elif rate_dict[sname] < 0:
            state_color.append(RdBu9[1]) # if more retweets of votehillary, mark as blue
            state_rates.append(rate_dict[sname])
        else:
            state_color.append(RdGy9[3])
            state_rates.append(0)
    except:
        state_color.append(RdGy9[3])
        state_rates.append(0)

# Bokeh source data dictionary
source = ColumnDataSource(data=dict(
    lon=state_xs,
    lat=state_ys,
    name=state_names,
    rate=state_rates,
    fill_color=state_color,
))

# set figure
TOOLS = "pan,wheel_zoom,box_zoom,reset,hover,save"

map_p = figure(
    title="US Election 2016 'Poll'", tools=TOOLS,
    x_axis_location=None, y_axis_location=None,
    plot_width=760, plot_height=440
)
map_p.grid.grid_line_color = None

# set data
map_p.patches("lon", "lat",source=source, # obtain data from source data dictionary
          fill_color="fill_color",
          fill_alpha=0.7, line_color="white", line_width=0.5)

# set hover information
hover = map_p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("State", "@name"),
    ("Rate", "@rate")
]

show(map_p)

Although low-level interface looks more complicated, it provides more attributes for you to configure. It means you can combine these features to create what exactly you want.

# Running a Bokeh Server
I believe you would like to explore the secret behind the data by yourself. Does retweet count is positively correlated to tweet count? You could try to find the relationships between data while interacting with Bokeh server.

Prior to creating a server application, do not forget to open server. 

    $bokeh serve

In [115]:
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.io import curdoc, output_notebook, push_notebook, show

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

output_notebook()

Loading BokehJS ...

In [116]:
# mapping of display name and dataframe column name
axis_map = {
    "Retweet Count": "tweet_retweet_count",
    "Favorite Count": "tweet_favorite_count",
    "Tweet Count": "tweet_count",
}

# source data dictionary
source = ColumnDataSource(data=
            dict(x=grouped_vote["tweet_count"], y=grouped_vote["tweet_retweet_count"]))

# configure figure
TOOLS = "pan,box_zoom,wheel_zoom,save"
interact_scatter = figure(tools=TOOLS, plot_width=400, plot_height=400,
                          title="Adhoc Analyze Poll 2016")

# configure plot
interact_scatter.circle(x="x", y="y", source=source)

# plot update function
# be called during interacting
def update(x_axis="Tweet Count", y_axis="Retweet Count"):
    # get data column name
    x_name = axis_map[x_axis]
    y_name = axis_map[y_axis]
    # change axis label display name
    interact_scatter.xaxis.axis_label = x_axis
    interact_scatter.yaxis.axis_label = y_axis
    # change data column
    source.data = dict(
        x=grouped_vote[x_name],
        y=grouped_vote[y_name],
    )
    # push plot updates to notebook
    # if you use html to show the results, this part will be very different
    push_notebook()

# function order: output_notebook(), show(plot, notebook_handle=True), push_notebook()
# cannot push before show, change the order will cause compile issue
show(interact_scatter, notebook_handle=True)


In [118]:
interact(update,
         x_axis=["Retweet Count", "Favorite Count", "Tweet Count"], # dropdown list
         y_axis=["Retweet Count", "Favorite Count", "Tweet Count"])

<function __main__.update>

Now, you can change the data on x or y axis by yourself and explore their relationships. Try it! 

If you are interested in making a more fancy interact platform, you can work on html/js output and you will find bokeh.layouts and widgetbox may help you a lot. 


Hope you enjoy this tutorial.
Thank you!

# References
1. Bokeh: http://bokeh.pydata.org/en/latest/
2. Twitter API: https://dev.twitter.com/
3. Tweepy: http://www.tweepy.org/
4. [Average Latitude and Longitude for US States](http://dev.maxmind.com/geoip/legacy/codes/state_latlon/)
5. [Sample Bounding Boxes csv File for USA Counties](https://raw.githubusercontent.com/stucka/us-county-bounding-boxes/master/bounding.csv)
6. Interactive Queries [Server App Example](https://github.com/bokeh/bokeh/tree/master/examples/app/movies)